
Name: Rachel Fanti Coelho Lima    
Date: 02/2022     
Subject: LM-18 73005B - Data Profiling - AY 2021-22    
Task: PLI - duplicated.


In [4]:
%matplotlib inline

In [5]:
# pip install PyQt5==5.9.2

In [6]:
import time
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn
import sys

#pip install pandas_profiling --user
#pip install PyQt5==5.9.2
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

### Input Data 

In [7]:
#df = pd.DataFrame([ [ 'C', '3', 'X', 722, 112 ],
#                    [ 'A', '1', 'X', 289, 553 ],
#                    [ 'A', '1', 'Y', 189, 583 ],
#                    [ 'B', '1', 'X', 289, 513 ],
#                    [ 'C', '1', 'X', 289, 553 ] ], columns = [a for a in 'abcde'])

In [9]:
df = pd.read_csv('./../00_data/Consolidado.csv', sep = ",", index_col=False)
df.columns=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

In [10]:
df.head()

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p
0,Taurasi,Taurasi,Taurasi,NaN,Agricola Boccella,Color rubino limpido; l’olfatto è subito inten...,100% Aglianico,"14,5%",D.O.C.G.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,www.vinix.com
1,Casefatte,Campania,Bianco,NaN,Agricola Boccella,"Di colore giallo paglierino, profumi agrumati,...",100% Fiano,13%,I.G.T.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,http://www.doctorwine.it/
2,Rasott,Irpinia Campi Taurasini,Rosso,NaN,Agricola Boccella,Color rubino limpido; l’olfatto è subito inten...,100% Aglianico,14%,D.O.C.,"Villaggio S. Eustachio, 16",Castelfranci,Avellino,"40.927652, 15.052481",http://www.boccellavini.it/,0827 72574,www.vinix.com
3,Sannio Falanghina,Falanghina del Sannio,Bianco,NaN,Agricola Irpina,Colore giallo paglierino con riflessi verdogno...,100% Falanghina,NaN,D.O.C.,"Via Limaturo, 48",San Michele di Pratola Serra,Avellino,"40.987214, 14.865230",http://www.agricolairpina.com/it/chi_siamo.php,0825 967038,NaN
4,Irpinia Aglianico,Irpinia,Rosso,NaN,Agricola Irpina,"Di colore rosso rubino, ha profumo intenso, sa...",85-90% Aglianico; 15-10% Merlot o Sangiovese,NaN,D.O.C.,"Via Limaturo, 48",San Michele di Pratola Serra,Avellino,"40.987214, 14.865230",http://www.agricolairpina.com/it/chi_siamo.php,0825 967038,NaN


In [107]:
schema = set(a for a in df.columns)
#schema

### Helper functions

In [108]:
def sort_single(s):
    return ''.join(sorted(list(s)))

def sort(F):
    return sorted(sort_single(s) for s in F)

In [109]:
def pli_single(a, D):
    pli = {}
    for row in range(D.shape[0]):
        key = D.at[row, a]
        if key in pli:
            pli[key].add(row)
        else:
            pli[key] = { row }
    return list(v for v in pli.values() if len(v) > 1)

In [110]:
def pli_intersect(p1, p2):
    pli, prob = {}, {}
    for c, s in enumerate(p1):
        for row in s:
            prob[row] = c
    for c, s in enumerate(p2):
        for row in s:
            if row in prob:
                key = (c, prob[row])
                if key in pli:
                    pli[key].add(row)
                else:
                    pli[key] = { row }
            
    return list(v for v in pli.values() if len(v) > 1)

### Duplicated algorithm (PLI)

In [111]:
table = df

In [112]:
pli = {a: pli_single(a, table) for a in schema}
#pli

In [114]:
k=2
while k<= len (df.columns):
    columns_pli = [i for i in df.columns[0:k]]
    pli [sort_single(columns_pli)] = pli_intersect(pli[sort_single(columns_pli[:-1])], pli[columns_pli[-1]])
    #= pli_intersect(pli['ab'], pli['d'])
    k = k+1
print ('pli', sort_single(df.columns))
print (pli[sort_single(df.columns)])

pli abcdefghijklmnop
[{4, 1863}, {1864, 5}, {1865, 6}, {9, 1868}, {12, 1871}, {1872, 13}, {1873, 14}, {1874, 15}, {17, 1876}, {19, 3981, 1878}, {32, 1891}, {1896, 37}, {1897, 38}, {4003, 39}, {44, 1903}, {1913, 54}, {56, 1915}, {57, 1916}, {59, 1918}, {60, 1919}, {1920, 61}, {1921, 62}, {64, 1923}, {65, 4009, 1924}, {67, 1926}, {68, 1927}, {1929, 70}, {73, 1932}, {74, 1933}, {75, 1934}, {4028, 76}, {1937, 78}, {1944, 85}, {1946, 87}, {88, 1947}, {89, 1948}, {91, 1950}, {1952, 93}, {1953, 94}, {96, 1955}, {97, 1956}, {98, 1957}, {99, 1958}, {100, 1959}, {1960, 101}, {1961, 102}, {104, 1963}, {105, 1964}, {106, 1965}, {1986, 4039, 127}, {128, 4040}, {129, 4038}, {132, 1991}, {1992, 133}, {1994, 135}, {136, 1995}, {137, 1996}, {138, 1997}, {139, 1998}, {140, 1999}, {2000, 141}, {2001, 142}, {144, 2003}, {145, 2004}, {3777, 146, 2005}, {147, 3772, 2006}, {148, 2007}, {2008, 3779, 149}, {161, 2020}, {162, 2021}, {163, 2022}, {164, 2023}, {2024, 165}, {2025, 166}, {2026, 167}, {169, 2028}, {

In [117]:
count_duplicates = 0
for i in pli[sort_single(df.columns)]:
    count_duplicates += (len (i) -1)
print (count_duplicates)

1089
